In [106]:
import os
import random
import itertools
import numpy as np
import cv2
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import xml.etree.ElementTree as et

In [107]:
os.chdir('C:\\Users\\Timur\\Documents\\python_projects\\Comvis\\generation_data\\data')

In [108]:
def box_points(img_shape, points):
    for i in range(len(points)):
        if i % 2 == 0:
            points[i] = int(points[i])/(img_shape[1]/256)
        else:
            points[i] = int(points[i])/(img_shape[0]/256)
    return points

In [109]:
seq0 = iaa.OneOf(
    [
        iaa.Snowflakes(flake_size= 0.6, speed=0.06),
        iaa.Clouds(1),
        iaa.RainLayer(density=(0.3,0.4), density_uniformity = (0.5,0.7), drop_size = (0.3, 0.4), drop_size_uniformity = 0.5, angle = 2, speed = (0.2, 0.4), blur_sigma_fraction = 0.9)
    ] )

In [110]:
seq1 = iaa.OneOf(
    [
        iaa.GaussianBlur(sigma= (0.7 ,1.2)),
        iaa.LinearContrast((0.6,1.3)),
        iaa.Multiply((0.9,1.2), per_channel=0.2)
    ])

In [111]:
seq = iaa.SomeOf(2,
    [
        seq1,
        seq0,
        iaa.Fliplr(1),
        iaa.Sometimes(0.5, iaa.Affine(rotate=(-5,5))),
        iaa.Sometimes(0.4, iaa.Crop(percent=(0, 0.05)))
    ])

In [112]:
def augmentation(img, bbs, n, file_name, img_name):
    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)
    for i in range(len(bbs_aug.bounding_boxes)):
        new_box = bbs_aug.bounding_boxes[i]
    img_path = f'{n}.jpg'
    cv2.imwrite(img_path, image_aug)
    os.chdir('../aug_labels')
    my_file = open(f"{n}.txt", "w+")
    my_file.write(f"{file_name}\n{img_name}\n{new_box.x1}, {new_box.y1}, {new_box.x2}, {new_box.y2}")
    my_file.close()

In [113]:
for item in os.listdir():
    os.chdir(f'C:\\Users\\Timur\\Documents\\python_projects\\Comvis\\generation_data\\data\\{item}\\labels')
    os.mkdir('../aug_images')
    os.mkdir('../aug_labels')
    count = 0
    for img_label in os.listdir():
        bb = []
        bb_points = []
        count += 1
        tree = et.parse(img_label)
        file_name = tree.find('filename').text
        img_name = tree.find('object/name').text
        boxes = tree.findall('object/bndbox')
        for box in boxes:
            for elem in box:
                bb_points.append(elem.text) 
        os.chdir('../images')
        image = cv2.imread(file_name)
        bb_points = box_points(image.shape, bb_points)
        os.chdir('../aug_images')
        for i in range(len(bb_points)//4): 
            bb.append(BoundingBox(x1=bb_points[i*4-4], y1=bb_points[i*4-3], x2=bb_points[i*4-2], y2=bb_points[i*4-1]))
        bbs = BoundingBoxesOnImage(bb, shape=(256,256))
        image = cv2.resize(image, (256,256))
        augmentation(image, bbs, count, file_name, img_name)
        os.chdir('../labels')
    os.chdir(f'C:\\Users\\Timur\\Documents\\python_projects\\Comvis\\generation_data\\data\\')

AttributeError: 'NoneType' object has no attribute 'text'